In [4]:
import pandas as pd
import pathlib

# 1. 路径（保持你原来的）
baseline_path = pathlib.Path(r"C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 2\outcom\半月板手术_outcome_baseline.xlsx")
one_yr_path   = pathlib.Path(r"C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 2\outcom\半月板手术_outcome_3y.xlsx")
out_path      = pathlib.Path(r"C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 2\半月板手术_outcome_3y_processed.xlsx")

# 2. 读 Excel
bl  = pd.read_excel(baseline_path)
one = pd.read_excel(one_yr_path)

# 3. 只保留评分列
cols = ['WOMKP', 'WOMSTF', 'WOMADL', 'WOMTS', 'KOOSKP', 'KOOSYM']
bl  = bl[cols]
one = one[cols].add_suffix('_1y')

# 4. 行对行拼接
df = pd.concat([bl, one], axis=1)

# 5. 各自阈值，生成二分类列（列名先叫 xxx_improved）
threshold = {
    'WOMKP' : -3.99,
    'WOMSTF': 2,
    'WOMADL': 3,
    'WOMTS' : 2,
    'KOOSKP': 5,
    'KOOSYM': 4
}

for col in threshold:
    delta_col = f'{col}_delta'
    flag_col  = f'{col}_improved'
    df[delta_col] = df[f'{col}_1y'] - df[col]
    df[flag_col]  = (df[delta_col] < threshold[col]).astype(int)

# 6. 只保留二分类列，并把列名恢复成原始评分名
flag_cols = [f'{c}_improved' for c in threshold]   # 现成的二分类列
out_df = df[flag_cols].rename(columns=dict(zip(flag_cols, threshold.keys())))

# 7. 保存为 Excel
out_df.to_excel(out_path, index=False)
print('处理完成，已保存为：', out_path.resolve())

处理完成，已保存为： C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 2\半月板手术_outcome_3y_processed.xlsx
